# Scraping

In [ ]:
import requests
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
data = requests.get(standings_url)
from bs4 import BeautifulSoup
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]
team_urls = [f"https://fbref.com{l}" for l in links]
data = requests.get(team_urls[0])
import pandas as pd
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]
data = requests.get(f"https://fbref.com{links[0]}")
shooting = pd.read_html(data.text, match="Shooting")[0]
shooting.head()

<ipython-input-8-eb33c46a5dcb>:13: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
<ipython-input-8-eb33c46a5dcb>:19: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]


For Liverpool                                                              \
           Date   Time            Comp         Round  Day Venue Result   GF   
0    2024-08-17  12:30  Premier League   Matchweek 1  Sat  Away      W  2.0   
1    2024-08-25  16:30  Premier League   Matchweek 2  Sun  Home      W  2.0   
2    2024-09-01  16:00  Premier League   Matchweek 3  Sun  Away      W  3.0   
3    2024-09-14  15:00  Premier League   Matchweek 4  Sat  Home      L  0.0   
4    2024-09-17  21:00    Champions Lg  League phase  Tue  Away      W  3.0   

                         ... Standard               Expected               \
    GA         Opponent  ...     Dist   FK PK PKatt       xG npxG npxG/Sh   
0  0.0     Ipswich Town  ...     14.8  0.0  0     0      2.6  2.6    0.15   
1  0.0        Brentford  ...     13.6  1.0  0     0      2.5  2.5    0.14   
2  0.0   Manchester Utd  ...     13.4  0.0  0     0      1.8  1.8    0.16   
3  1.0  Nott'ham Forest  ...     14.9  0.0  0     0      0.9  0.9    0.07   
4  1.0         it Milan  ...     15.7  1.0  0     0      3.1  3.1    0.14   

               Unnamed: 25_level_0  
  G-xG np:G-xG        Match Report  
0 -0.6    -0.6        Match Report  
1 -0.5    -0.5        Match Report  
2  1.2     1.2        Match Report  
3 -0.9    -0.9        Match Report  
4 -0.1    -0.1        Match Report  

[5 rows x 26 columns]

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# URL mới nhất của Premier League
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

# Danh sách để lưu trữ dữ liệu từ tất cả các đội
all_matches = []

# Lấy dữ liệu bảng xếp hạng hiện tại
data = requests.get(standings_url)
soup = BeautifulSoup(data.text, "html.parser")

# Lấy link của từng đội bóng từ bảng xếp hạng
standings_table = soup.select('table.stats_table')[0]
links = [l.get("href") for l in standings_table.find_all('a')]
links = [l for l in links if '/squads/' in l]
team_urls = [f"https://fbref.com{l}" for l in links]

# Lặp qua các đội bóng để lấy dữ liệu
for team_url in team_urls:
    team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
    print(f"Đang lấy dữ liệu cho đội: {team_name}")

    # Lấy dữ liệu trận đấu "Scores & Fixtures"
    team_response = requests.get(team_url)
    matches = pd.read_html(team_response.text, match="Scores & Fixtures")[0]

    # Lấy link Shooting Stats
    soup = BeautifulSoup(team_response.text, "html.parser")
    links = [l.get("href") for l in soup.find_all('a') if l.get("href") and 'all_comps/shooting/' in l]

    if not links:  # Kiểm tra nếu không tìm thấy link Shooting
        print(f"Không tìm thấy Shooting Stats cho đội {team_name}")
        continue

    shooting_url = f"https://fbref.com{links[0]}"
    shooting_response = requests.get(shooting_url)
    shooting = pd.read_html(shooting_response.text, match="Shooting")[0]
    shooting.columns = shooting.columns.droplevel()

    # Kết hợp dữ liệu Matches và Shooting
    try:
        team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
    except ValueError:
        print(f"Lỗi khi kết hợp dữ liệu cho đội {team_name}")
        continue

    # Lọc chỉ dữ liệu Premier League
    team_data = team_data[team_data["Comp"] == "Premier League"]
    team_data["Team"] = team_name
    all_matches.append(team_data)

    time.sleep(1)  # Tránh gửi quá nhiều request liên tục

# Kết hợp tất cả dữ liệu từ các đội thành DataFrame
if all_matches:
    match_df = pd.concat(all_matches, ignore_index=True)
    match_df.columns = [c.lower() for c in match_df.columns]
    print("Dữ liệu tổng hợp thành công!")
    print(match_df)
else:
    print("Không có dữ liệu nào được thu thập.")



In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL của trang thống kê Premier League
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

# Gửi yêu cầu HTTP GET
data = requests.get(standings_url)

# Phân tích nội dung HTML với BeautifulSoup
soup = BeautifulSoup(data.text, 'html.parser')

# Tìm bảng Squad Standard Stats bằng id 'stats_squads_standard_for'
table = soup.find('table', {'id': 'stats_squads_standard_for'})

# Nếu bảng được tìm thấy, tiến hành xử lý
if table:
    # Đọc dữ liệu từ bảng và chuyển thành DataFrame
    df = pd.read_html(str(table))[0]

    # Lưu dữ liệu vào CSV
    file_path = '/content/drive/My Drive/Excel/stats_squads_standard_for.csv'
    df.to_csv(file_path, index=False)
    print("Dữ liệu đã được lưu vào file stats_squads_shooting.csv")
else:
    print("Không tìm thấy bảng Squad Standard Stats.")

Dữ liệu đã được lưu vào file stats_squads_shooting.csv


<ipython-input-48-72bfb8ce161c>:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


# Train model

In [ ]:
import pandas as pd
import numpy as np
import pickle  # Import thư viện pickle để lưu và tải mô hình

# 1. Đọc dữ liệu
matches = pd.read_csv("/content/drive/MyDrive/Excel/data_stats/premier_league_matches.csv")  # Dataset kết quả trận đấu
stats = pd.read_csv("/content/drive/MyDrive/Excel/data_stats/stats_squads_standard.csv")  # Dataset chỉ số trung bình

# 2. Lấy chỉ số trung bình cho một đội bóng
def get_team_stats(team_name, stats_df):
    team_data = stats_df[stats_df['Squad'] == team_name]
    if team_data.empty:
        raise ValueError(f"Không tìm thấy dữ liệu cho đội: {team_name}")
    return team_data['xG'].mean(), team_data['xAG'].mean(), team_data['Poss'].mean()

# 3. Chuẩn bị dữ liệu train
def prepare_training_data(matches_df, stats_df):
    X = []
    y = []
    for _, row in matches_df.iterrows():
        try:
            # Chỉ số của team và opponent
            team_xg, team_xga, team_poss = get_team_stats(row['team'], stats_df)
            opp_xg, opp_xga, opp_poss = get_team_stats(row['opponent'], stats_df)

            # Tính chênh lệch chỉ số giữa team và opponent
            diff_xg = team_xg - opp_xga
            diff_xga = team_xga - opp_xg
            diff_poss = team_poss - opp_poss

            # Đưa vào input X
            X.append([diff_xg, diff_xga, diff_poss])

            # Kết quả trận đấu: W -> 0, D -> 1, L -> 2
            if row['result'] == 'W':
                y.append(0)  # Win
            elif row['result'] == 'L':
                y.append(2)  # Lose
            else:  # Draw
                y.append(1)
        except ValueError as e:
            print(e)
    return np.array(X), np.array(y)

# 4. Xây dựng Logistic Regression từ đầu
class LogisticRegressionCustom:
    def __init__(self, lr=0.01, epochs=1000):
        self.lr = lr
        self.epochs = epochs
        self.weights = None
        self.bias = None

    def softmax(self, z):
        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))  # Prevent overflow
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)

    def fit(self, X, y):
        n_samples, n_features = X.shape
        n_classes = 3  # Win, Draw, Lose
        self.weights = np.zeros((n_features, n_classes))
        self.bias = np.zeros(n_classes)

        # One-hot encode y
        y_onehot = np.zeros((n_samples, n_classes))
        y_onehot[np.arange(n_samples), y] = 1

        # Gradient Descent
        for _ in range(self.epochs):
            linear_model = np.dot(X, self.weights) + self.bias
            y_pred = self.softmax(linear_model)

            # Compute gradients
            dw = (1 / n_samples) * np.dot(X.T, (y_pred - y_onehot))
            db = (1 / n_samples) * np.sum(y_pred - y_onehot, axis=0)

            # Update weights and bias
            self.weights -= self.lr * dw
            self.bias -= self.lr * db

    def predict_proba(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        return self.softmax(linear_model)

    def predict(self, X):
        probs = self.predict_proba(X)
        return np.argmax(probs, axis=1)

    # Phương thức lưu model
    def save_model(self, file_path):
        with open(file_path, 'wb') as f:
            pickle.dump({'weights': self.weights, 'bias': self.bias}, f)
        print(f"Model đã được lưu vào {file_path}")

    # Phương thức tải model
    def load_model(self, file_path):
        with open(file_path, 'rb') as f:
            model_data = pickle.load(f)
            self.weights = model_data['weights']
            self.bias = model_data['bias']
        print(f"Model đã được tải từ {file_path}")

# 5. Dự đoán kết quả trận đấu
def predict_match(model, team, opponent, stats_df):
    try:
        team_xg, team_xga, team_poss = get_team_stats(team, stats_df)
        opp_xg, opp_xga, opp_poss = get_team_stats(opponent, stats_df)

        diff_xg = team_xg - opp_xga
        diff_xga = team_xga - opp_xg
        diff_poss = team_poss - opp_poss

        input_features = np.array([[diff_xg, diff_xga, diff_poss]])
        probs = model.predict_proba(input_features)[0]

        print(f"Xác suất thắng của {team}: {probs[0] * 100:.2f}%")
        print(f"Xác suất hòa: {probs[1] * 100:.2f}%")
        print(f"Xác suất thắng của {opponent}: {probs[2] * 100:.2f}%")

        if np.argmax(probs) == 0:
            return f"Đội {team} được dự đoán thắng"
        elif np.argmax(probs) == 2:
            return f"Đội {opponent} được dự đoán thắng"
        else:
            return f"Trận đấu được dự đoán hòa"

    except ValueError as e:
        return str(e)

# 6. Train và sử dụng mô hình
if __name__ == "__main__":
    model_file = "logistic_model.pkl"

    # Chuẩn bị dữ liệu train
    X_train, y_train = prepare_training_data(matches, stats)

    # Khởi tạo và train Logistic Regression
    model = LogisticRegressionCustom(lr=0.01, epochs=2000)
    model.fit(X_train, y_train)

    # Lưu model sau khi train
    model.save_model(model_file)



In [ ]:
# Tải lại model để kiểm tra
loaded_model = LogisticRegressionCustom()
loaded_model.load_model(model_file)

# Dự đoán trận đấu mới
team = "Aston Villa FC"
opponent = "Manchester City FC"

prediction = predict_match(loaded_model, team, opponent, stats)
print(prediction)

Model đã được tải từ logistic_model.pkl
Xác suất thắng của Aston Villa FC: 40.39%
Xác suất hòa: 27.69%
Xác suất thắng của Manchester City FC: 31.92%
Đội Aston Villa FC được dự đoán thắng


# Đánh giá tập test

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Dự đoán kết quả cho tập test
y_pred = model.predict(X_test)

# In ra báo cáo đánh giá mô hình
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Vẽ confusion matrix
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Win", "Draw", "Lose"], yticklabels=["Win", "Draw", "Lose"])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()

